## 트라이(Trie)
- ![Alt text](assets/trie-1.png)
- 사전에서 'cancel'이라는 단어를 찾을 때 c의 색인을 찾고 그다음 a의 색인을 찾고 n -> c... 이런식으로 찾는다. 이러한 순서를 컴퓨터에 적용한 방식이 트라이 구조이다.
- 트라이 상의 가장 긴 문자열을 $S$, 문자열의 개수를 $N$이라고 할 때
  - 트라이를 구축하는 시간복잡도는 $O(LN)$이다.
  - 트라이에 추가하는 시간복잡도는 $O(S)$이다.
  - 트라이에서 문자열을 찾는 시간복잡도는 $O(S)$이다.


### 트라이의 동작
- 트라이로 구성된 트리 구조를 `trie`, 추가하고자 하는 문자열을 `p`라고 하고, 시작 지점은 루트노드이다
  - p의 `i`번째 글자인 `p[i]`으로 가는 간선이 현재 노드에 존재하는지 확인한다. 
    - 존재한다면 `p[i]`로 가는 간선을 따라 다음노드로 이동한다.
    - 존재하지 않는다면 현재 노드에서 `p[i]`로 가는 노드와 간선을 만들고 해당 노드로 이동한다.
  - `i`가 `len(p)`가 될 때까지 위 과정을 반복한다.

### 구현
- 주석은 5052(전화번호 목록) 참고

In [13]:
class Trie:
  def __init__(self):
    self.root = {}

  def add(self, s):
    cur = self.root
    for c in s:
      cur = cur.setdefault(c, {})
    cur["_end_"] = True

  def __delitem__(self, s):
    cur = self.root
    S = [cur]
    for c in s:
      cur = cur[c]
      S.append(cur)
    del cur["_end_"]

  def query(self, s): #implement this
    cur = self.root
    for c in s:
      if c not in cur:
        return False
      cur = cur[c]
    return "_end_" in cur

### 비트마스크와 트라이
- 나올 수 있는 문자열이 0과 1로만 이루어져 있기 때문에 비트마스크를 이용하여 트라이를 구현할 수 있다.
- 특히 XOR 관련 문제에서 많이 나온다.

### 이진 트라이 구현
- 쿼리는 xor했을 때 mex를 구한다. 주석은 16902(mex) 참고

In [ ]:
class Node:
  def __init__(self):
    self.child = [None, None]
    self.n = 0
  
  def __contains__(self, k: bool):
    return self.child[k] is not None
  
  def __getitem__(self, k: bool):
    if self.child[k] is not None: return self.child[k]
    self.child[k] = Node()
    return self.child[k]

  def add(self, k: bool) :
    if self.child[k] is not None : return
    self.child[k] = Node() 

class BinaryTrie:
  def __init__(self, depth=32):
    self.root = Node()
    self.depth = depth

  def __contains__(self, n) :
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if v not in cur : return False
      cur = cur[v]
    return True

  def add(self, n): #assume no duplicates, O(self.depth)
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      cur.add(v)
      cur.n += 1
      cur = cur[v]
    cur.n += 1

  def __delitem__(self, n) : #O(self.depth)
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if v not in cur : return False
      cur.n -= 1

      if cur.child[v].n == 1 :
        cur.child[v] = None
        return True
    
      cur = cur[v]
    return True

  def __repr__(self): #print all value in decesnding order. O(self.depth * 2^self.depth)
    cur = self.root
    S = [(cur, 0, 0)]
    res = []
    while S :
      u, d, x = S.pop()
      if u.n == 1 and d == self.depth:
        res.append(str(x))
        continue
        
      for i, v in enumerate(u.child) :
        if not v : continue
        S.append((v, d+1, x*2+i))
      
    return f"[{' '.join(res)}]"

  def query(self, n):
    cur = self.root
    res = 0
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if not cur.child[v] :
        break
      elif cur.child[v].n == (1 << i) :
        cur = cur[not v] 
        res += 1 << i
      else :
        cur = cur[v]
    return res